In [1]:
import json

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

from pydantic import BaseModel
from typing import List, Optional

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from supabase import create_client

In [2]:
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service("/usr/bin/chromedriver")

driver = webdriver.Chrome(service=service, options=chrome_options)

In [3]:
whoscored_url = "https://www.whoscored.com/Matches/1821197/Live/England-Premier-League-2024-2025-Chelsea-Newcastle"

driver.get(whoscored_url)

In [4]:
soup = BeautifulSoup(driver.page_source,'html.parser')

In [5]:
element = soup.select_one('script:-soup-contains("matchCentreData")')

In [6]:
matchdict = json.loads(element.text.split("matchCentreData: ")[1].split(",\n")[0])

In [8]:
matchdict

{'playerIdNameDictionary': {'369430': 'Enzo Fernández',
  '425319': 'Roméo Lavia',
  '404749': 'Filip Jørgensen',
  '410175': 'Moisés Caicedo',
  '373946': 'Wesley Fofana',
  '300945': 'Christopher Nkunku',
  '403850': 'Malo Gusto',
  '434077': 'Renato Veiga',
  '337916': 'Pedro Neto',
  '362431': 'João Félix',
  '426050': 'Nicolas Jackson',
  '363496': 'Marc Cucurella',
  '362151': 'Robert Sánchez',
  '409376': 'Mykhailo Mudryk',
  '421351': 'Levi Colwill',
  '389935': 'Noni Madueke',
  '395692': 'Cole Palmer',
  '136464': 'Tosin Adarabioyo',
  '346300': 'Jadon Sancho',
  '361330': 'Reece James',
  '201755': 'Joelinton',
  '338780': 'Bruno Guimarães',
  '416209': 'William Osula',
  '110273': 'Odysseas Vlachodimos',
  '343501': 'Sandro Tonali',
  '303924': 'Miguel Almirón',
  '100599': 'Emil Krafth',
  '299254': 'Alexander Isak',
  '82726': 'Fabian Schär',
  '105720': 'Nick Pope',
  '8643': 'John Ruddy',
  '82277': 'Dan Burn',
  '141486': 'Jacob Murphy',
  '331382': 'Harvey Barnes',
  

In [209]:
matchdict.keys()

dict_keys(['playerIdNameDictionary', 'periodMinuteLimits', 'timeStamp', 'attendance', 'venueName', 'referee', 'weatherCode', 'elapsed', 'startTime', 'startDate', 'score', 'htScore', 'ftScore', 'etScore', 'pkScore', 'statusCode', 'periodCode', 'home', 'away', 'maxMinute', 'minuteExpanded', 'maxPeriod', 'expandedMinutes', 'expandedMaxMinute', 'periodEndMinutes', 'commonEvents', 'events', 'timeoutInSeconds'])

In [7]:
match_events = matchdict["events"]
match_events

[{'id': 2740018849.0,
  'eventId': 3,
  'minute': 0,
  'second': 0,
  'teamId': 23,
  'x': 0.0,
  'y': 0.0,
  'expandedMinute': 0,
  'period': {'value': 1, 'displayName': 'FirstHalf'},
  'type': {'value': 32, 'displayName': 'Start'},
  'outcomeType': {'value': 1, 'displayName': 'Successful'},
  'qualifiers': [],
  'satisfiedEventsTypes': [],
  'isTouch': False},
 {'id': 2740018839.0,
  'eventId': 3,
  'minute': 0,
  'second': 0,
  'teamId': 15,
  'x': 0.0,
  'y': 0.0,
  'expandedMinute': 0,
  'period': {'value': 1, 'displayName': 'FirstHalf'},
  'type': {'value': 32, 'displayName': 'Start'},
  'outcomeType': {'value': 1, 'displayName': 'Successful'},
  'qualifiers': [],
  'satisfiedEventsTypes': [],
  'isTouch': False},
 {'id': 2740018879.0,
  'eventId': 4,
  'minute': 0,
  'second': 0,
  'teamId': 15,
  'playerId': 395692,
  'x': 50.0,
  'y': 50.0,
  'expandedMinute': 0,
  'period': {'value': 1, 'displayName': 'FirstHalf'},
  'type': {'value': 1, 'displayName': 'Pass'},
  'outcomeType

In [8]:
df = pd.DataFrame(match_events)
df.tail()

,id,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,endY,relatedEventId,relatedPlayerId,blockedX,blockedY,goalMouthZ,goalMouthY,isShot,cardType,isGoal
1544,2.740161e+09,889,98,27.0,15,0.0,0.0,106,"{'value': 2, 'displayName': 'SecondHalf'}","{'value': 30, 'displayName': 'End'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1545,2.740162e+09,817,0,0.0,23,0.0,0.0,16,"{'value': 14, 'displayName': 'PostGame'}","{'value': 30, 'displayName': 'End'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1546,2.740161e+09,890,0,0.0,15,0.0,0.0,16,"{'value': 14, 'displayName': 'PostGame'}","{'value': 30, 'displayName': 'End'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1547,2.739963e+09,2,0,0.0,15,0.0,0.0,0,"{'value': 16, 'displayName': 'PreMatch'}","{'value': 34, 'displayName': 'FormationSet'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1548,2.739963e+09,2,0,0.0,23,0.0,0.0,0,"{'value': 16, 'displayName': 'PreMatch'}","{'value': 34, 'displayName': 'FormationSet'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [223]:
df.columns

Index(['id', 'eventId', 'minute', 'second', 'teamId', 'x', 'y',
       'expandedMinute', 'period', 'type', 'outcomeType', 'qualifiers',
       'satisfiedEventsTypes', 'isTouch', 'playerId', 'endX', 'endY',
       'relatedEventId', 'relatedPlayerId', 'blockedX', 'blockedY',
       'goalMouthZ', 'goalMouthY', 'isShot', 'cardType', 'isGoal'],
      dtype='object')

In [10]:
df["teamName"] = df["teamId"].apply(lambda x: matchdict["home"]["name"] if x == matchdict["home"]["teamId"] else matchdict["away"]["name"])

In [12]:
df.columns

Index(['id', 'eventId', 'minute', 'second', 'teamId', 'x', 'y',
       'expandedMinute', 'period', 'type', 'outcomeType', 'qualifiers',
       'satisfiedEventsTypes', 'isTouch', 'playerId', 'endX', 'endY',
       'relatedEventId', 'relatedPlayerId', 'blockedX', 'blockedY',
       'goalMouthZ', 'goalMouthY', 'isShot', 'cardType', 'isGoal', 'teamName'],
      dtype='object')

In [13]:
df.dropna(subset="playerId",inplace=True)
df["playerName"] = df["playerId"].apply(lambda x: matchdict["playerIdNameDictionary"][f"{int(x)}"] if matchdict["playerIdNameDictionary"][f"{int(x)}"] else x)
df.tail()

,id,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,relatedPlayerId,blockedX,blockedY,goalMouthZ,goalMouthY,isShot,cardType,isGoal,teamName,playerName
1538,2.740161e+09,886,97,40.0,15,3.0,46.8,105,"{'value': 2, 'displayName': 'SecondHalf'}","{'value': 6, 'displayName': 'CornerAwarded'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chelsea,Reece James
1539,2.740161e+09,814,98,19.0,23,99.3,99.3,106,"{'value': 2, 'displayName': 'SecondHalf'}","{'value': 1, 'displayName': 'Pass'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Newcastle,Jacob Murphy
1540,2.740161e+09,887,98,20.0,15,3.7,40.4,106,"{'value': 2, 'displayName': 'SecondHalf'}","{'value': 44, 'displayName': 'Aerial'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chelsea,Christopher Nkunku
1541,2.740162e+09,816,98,20.0,23,96.3,59.6,106,"{'value': 2, 'displayName': 'SecondHalf'}","{'value': 44, 'displayName': 'Aerial'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Newcastle,Nick Pope
1542,2.740161e+09,888,98,20.0,15,3.4,41.6,106,"{'value': 2, 'displayName': 'SecondHalf'}","{'value': 12, 'displayName': 'Clearance'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chelsea,Christopher Nkunku


In [14]:
df = df.where(pd.notnull(df), None)

In [15]:
df = df.rename(
    {
        'eventId': 'event_id',
        'expandedMinute': 'expanded_minute',
        'outcomeType': 'outcome_type',
        'isTouch': 'is_touch',
        'playerId': 'player_id',
        'teamId': 'team_id',
        'playerName': 'player_name',
        'teamName': 'team_name',
        'endX': 'end_x',
        'endY': 'end_y',
        'blockedX': 'blocked_x',
        'blockedY': 'blocked_y',
        'goalMouthZ': 'goal_mouth_z',
        'goalMouthY': 'goal_mouth_y',
        'isShot': 'is_shot',
        'cardType': 'card_type',
        'isGoal': 'is_goal'
    },
    axis=1
)

df.columns

Index(['id', 'event_id', 'minute', 'second', 'team_id', 'x', 'y',
       'expanded_minute', 'period', 'type', 'outcome_type', 'qualifiers',
       'satisfiedEventsTypes', 'is_touch', 'player_id', 'end_x', 'end_y',
       'relatedEventId', 'relatedPlayerId', 'blocked_x', 'blocked_y',
       'goal_mouth_z', 'goal_mouth_y', 'is_shot', 'card_type', 'is_goal',
       'team_name', 'player_name'],
      dtype='object')

In [16]:
df["period_display_name"] = df["period"].apply(lambda x: x["displayName"])
df["type_display_name"] = df["type"].apply(lambda x: x["displayName"])
df["outcome_type_display_name"] = df["outcome_type"].apply(lambda x: x["displayName"])

In [43]:
if "is_goal" not in df.columns:
    print("missing goals")
    df["is_goal"] = False

In [19]:
df["period_display_name"].head()

2    FirstHalf
3    FirstHalf
4    FirstHalf
5    FirstHalf
6    FirstHalf
Name: period_display_name, dtype: object

In [17]:
df.drop(columns=["period", "type", "outcome_type"], inplace=True)

In [18]:
df = df[[
    'id', 'event_id', 'minute', 'second', 'team_name', 'player_name', 'x', 'y', 'end_x', 'end_y',
    'qualifiers', 'is_touch', 'blocked_x', 'blocked_y', 'goal_mouth_z', 'goal_mouth_y', 'is_shot',
    'card_type', 'is_goal', 'type_display_name', 'outcome_type_display_name',
    'period_display_name'
]]

In [19]:
df.head(20)

,id,event_id,minute,second,team_name,player_name,x,y,end_x,end_y,...,blocked_x,blocked_y,goal_mouth_z,goal_mouth_y,is_shot,card_type,is_goal,type_display_name,outcome_type_display_name,period_display_name
2,2.740019e+09,4,0,0.0,Chelsea,Cole Palmer,50.0,50.0,18.4,49.9,...,NaN,NaN,NaN,NaN,None,None,None,Pass,Successful,FirstHalf
3,2.740019e+09,5,0,4.0,Chelsea,Robert Sánchez,20.7,52.7,64.4,30.7,...,NaN,NaN,NaN,NaN,None,None,None,Pass,Unsuccessful,FirstHalf
4,2.740019e+09,4,0,7.0,Newcastle,Dan Burn,28.6,74.5,42.9,66.2,...,NaN,NaN,NaN,NaN,None,None,None,Pass,Unsuccessful,FirstHalf
5,2.740019e+09,6,0,9.0,Chelsea,Roméo Lavia,60.2,36.7,78.0,19.9,...,NaN,NaN,NaN,NaN,None,None,None,Pass,Unsuccessful,FirstHalf
6,2.740019e+09,5,0,11.0,Newcastle,Lewis Hall,18.3,84.9,30.3,90.6,...,NaN,NaN,NaN,NaN,None,None,None,Pass,Successful,FirstHalf
7,2.740019e+09,6,0,12.0,Newcastle,Harvey Barnes,31.8,91.1,NaN,NaN,...,NaN,NaN,NaN,NaN,None,None,None,BallRecovery,Successful,FirstHalf
8,2.740019e+09,7,0,13.0,Newcastle,Harvey Barnes,31.8,91.1,26.4,79.6,...,NaN,NaN,NaN,NaN,None,None,None,Pass,Successful,FirstHalf
9,2.740019e+09,8,0,14.0,Newcastle,Joelinton,26.4,79.6,19.2,94.2,...,NaN,NaN,NaN,NaN,None,None,None,Pass,Successful,FirstHalf
10,2.740019e+09,9,0,16.0,Newcastle,Lewis Hall,19.2,94.2,30.6,96.9,...,NaN,NaN,NaN,NaN,None,None,None,Pass,Successful,FirstHalf
11,2.740019e+09,8,0,18.0,Chelsea,Noni Madueke,71.7,5.4,NaN,NaN,...,NaN,NaN,NaN,NaN,None,None,None,Challenge,Unsuccessful,FirstHalf


In [20]:
df.dtypes


id                           float64
event_id                       int64
minute                         int64
second                       float64
team_name                     object
player_name                   object
x                            float64
y                            float64
end_x                        float64
end_y                        float64
qualifiers                    object
is_touch                        bool
blocked_x                    float64
blocked_y                    float64
goal_mouth_z                 float64
goal_mouth_y                 float64
is_shot                       object
card_type                     object
is_goal                       object
type_display_name             object
outcome_type_display_name     object
period_display_name           object
dtype: object

In [21]:
df[['id', 'event_id', 'minute']] = df[['id', 'event_id', 'minute']].astype(np.int64)
df[['second', 'x', 'y', 'end_x', 'end_y']] = df[['second', 'x', 'y', 'end_x', 'end_y']].astype(float)
df[['is_shot', 'is_goal', 'card_type']] = df[['is_shot', 'is_goal', 'card_type']].astype(bool)

In [22]:
df['is_goal'] = df['is_goal'].fillna(False)
df['is_shot'] = df['is_shot'].fillna(False)

In [23]:
for column in df.columns:
    if df[column].dtype == np.float64 or df[column].dtype == np.float32:
        df[column] = np.where(
            np.isnan(df[column]),
            None,
            df[column]
        )

In [24]:
class MatchEvent(BaseModel):
    id: int
    event_id: int
    minute: int
    second: Optional[float] = None
    team_name: str
    player_name: str
    x: float
    y: float
    end_x: Optional[float] = None
    end_y: Optional[float] = None
    qualifiers: List[dict]
    is_touch: bool
    blocked_x: Optional[float] = None
    blocked_y: Optional[float] = None
    goal_mouth_z: Optional[float] = None
    goal_mouth_y: Optional[float] = None
    is_shot: bool
    card_type: bool
    is_goal: bool
    type_display_name: str
    outcome_type_display_name: str
    period_display_name: str

In [91]:
for x in df.to_dict(orient="records"):
    try:
        MatchEvent(**x).model_dump()
    except Exception as e:
        print(e)
        break

In [ ]:
supabase_password = ""
project_url = ""
api_key = ""

In [26]:
def insert_match_events(df,supabase):
    events = [
        MatchEvent(**x).model_dump()
        for x in df.to_dict(orient="records")
    ]

    execution = supabase.table("match_events").upsert(events).execute()

In [ ]:
supabase = create_client(project_url, api_key)
insert_match_events(df,supabase)